In [ ]:
{'Client ID': 'Syv7stn-Rd-mtrudAbRvuaqMUb_AgdSirVvOb9XofEnxQ4_oYlVzptzilTlFhbcq',
    'Client Secret':'A0fegVhhiemDPVDdDdtYkW-8j9GF-d6pFv4_-_sbuX7Hm3yOzqlPL-HRof0bzHYu9x-CaYc0ak9AMQ3ZIYt_Ig',
'Client Access Token': 'HIs4ia8HMYMSYsZVHrYSFdAhcum7gHC0ZWMPa5pXJOoQUhd86oe6maD-BkIcKrnz'


}

In [ ]:
export GENIUS_CLIENT_ACCESS_TOKEN="HIs4ia8HMYMSYsZVHrYSFdAhcum7gHC0ZWMPa5pXJOoQUhd86oe6maD-BkIcKrnz"

### Importing necessary modules and files

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
# loading data

df = pd.read_csv('lyrics.csv')
billboard = pd.read_csv('billboards.csv')

In [12]:
#df = pd.read_csv('lyrics.csv')
# after looking at the first element of the lyrics column, I need to be remove all the spacing and annotations in 
# the csv database as well as the scraped lyrics, because its not consistent with the other elements in the lyrics album
df['song'] = df.song.str.replace('-', " ")
df['artist'] = df.artist.str.replace('-', " ")
df['lyrics'] = df.lyrics.str.replace('\n', ' ')
df.head()

,index,song,year,artist,genre,lyrics
0,0,ego remix,2009,beyonce knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut..."
1,1,then tell me,2009,beyonce knowles,Pop,"playin' everything so easy, it's like you seem..."
2,2,honesty,2009,beyonce knowles,Pop,If you search For tenderness It isn't hard to ...
3,3,you are my rock,2009,beyonce knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a..."
4,4,black culture,2009,beyonce knowles,Pop,"Party the people, the people the party it's po..."


In [13]:
# cool after I finish cleaning this dataset, i need to figure out how to find the lyrics for the songs from my
# previous project. let me load the dataset into this project repo first

billboard = pd.read_csv('billboards.csv')
billboard.Song = billboard.Song.str.replace('>', 'than')
billboard.Artist = billboard.Artist.str.replace(' x ', ', ')
billboard.drop('num_weeks', inplace = True, axis = 1)
# billboard.head(20)

billboard_dict = billboard.to_dict('records')
billboard_dict[:5]


[{'Artist': '2 Chainz', 'Song': '4 AM'},
 {'Artist': '2 Chainz', 'Song': "It's A Vibe"},
 {'Artist': '2 Chainz', 'Song': 'Proud'},
 {'Artist': '2 Chainz, Gucci Mane, Quavo', 'Song': 'Good Drank'},
 {'Artist': '2 Chainz, Drake', 'Song': 'Bigger than You'}]

### Testing out the thought process to get to the final product

Notes:

1. punctuation like $ will be replaced by '-' in the url

1. multiple artists will be in the url, so maybe i need to get all of the artists from the spotify api? put in a try/except so that i can keep track of all the songs that didn't work and do that later for a smaller subset of songs



In [ ]:
from nltk import sent_tokenize, word_tokenize


# grabbing the data from the url
artist = 'Tessa Violet'
artist_url = artist.lower().replace(' ', '-')

source_code = requests.get(f'https://genius.com/{artist_url}-crush-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()

#cleaning

lyrics = lyrics.replace('\n', '. ')
sentences = sent_tokenize(lyrics)
cleaned = [sentence for sentence in sentences if ('bridge' not in sentence.lower()) and ('verse' not in sentence.lower()) and ('chorus' not in sentence.lower()) and len(sentence) >1]

data = [artist, song, cleaned]
key =['artist', 'song', 'lyrics']
song_dict = dict(zip(key, data))
print(song_dict)

In [ ]:

artist = 'Kanye West'
artist_url = artist.replace(' ', '-')

song = 'All of the Lights'
song_url = song.replace(' ', '-')


source_code = requests.get(f'https://genius.com/{artist_url}-{song_url}-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()

#cleaning

lyrics = lyrics.replace('\n', '. ')
sentences = sent_tokenize(lyrics)
cleaned = [sentence for sentence in sentences if 
           ('produced' not in sentence.lower()) and
           ('intro' not in sentence.lower()) and
           ('outro' not in sentence.lower()) and
           ('bridge' not in sentence.lower()) and 
           ('verse' not in sentence.lower()) and 
           ('chorus' not in sentence.lower()) and len(sentence) >1]
cleaned

#it would be nice to all of this in a dictionary that i can just put into an array and then turn into a pd dataframe

data = [artist, song, cleaned]
key =['artist', 'song', 'lyrics']
song_dict = dict(zip(key, data))
# print(song_dict) #yay it worked
# please note it takes like a second for one song...

In [ ]:
# https://genius.com/2-chainz-bigger-than-you-lyrics
    
source_code = requests.get('https://genius.com/2-chainz-bigger-than-you-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()
print((lyrics)) 

In [ ]:
billboard_dict

In [17]:
# i need to building a function that takes the billboard songs and just scrapes for songs!

def scrape_genius(artist, song):
    
    #artist = 'Kanye West'
    artist = artist.replace("'", '')
    song = song.replace("'", '')
    artist_url = artist.replace(' ', '-')
    #artist_url = artist.replace(',-', '-')
    #artist_url = artist.replace(',', '')

    #song = 'All of the Lights'
    song_url = song.replace(' ', '-')

    url = f'https://genius.com/{artist_url}-{song_url}-lyrics'
    # print(url)
    source_code = requests.get(url)
    soup = BeautifulSoup(source_code.text, 'html5lib')
    lyrics = soup.find('div', class_='lyrics').text.strip()

    #cleaning

    lyrics = lyrics.replace('\n', '. ')
    sentences = sent_tokenize(lyrics)
    cleaned = [sentence for sentence in sentences if 
               ('produced' not in sentence.lower()) and
               ('intro' not in sentence.lower()) and
               ('outro' not in sentence.lower()) and
               ('bridge' not in sentence.lower()) and 
               ('verse' not in sentence.lower()) and 
               ('chorus' not in sentence.lower()) and len(sentence) >1]
    cleaned

    #it would be nice to all of this in a dictionary that i can just put into an array and then turn into a pd dataframe

    data = [artist, song, cleaned]
    key =['artist', 'song', 'lyrics']
    song_dict = dict(zip(key, data))
    return song_dict

In [18]:
len(billboard_dict)

828

### Finally Scraping

In [19]:
smaller = billboard_dict[:20]

lyrics = []
missing = []
for record in smaller:
    song = record['Song']
    artist = record['Artist']
    artist = artist.replace(', ', ' ')
    artist = artist.replace('.', '')
    song = song.replace('.', '')
    #print(artist, song)
    # artist = record['Artist']
    try:
        lyrics.append(scrape_genius(artist, song))
    except:
        print(artist, song)
        missing.append(record)
        pass
    #print(lyrics)
    #print('done')

In [20]:
len(missing)

20

In [16]:
len(lyrics)

0

In [ ]:
billboard_lyrics = pd.DataFrame(lyrics)